## OpenAI의 Assistants API를 사용해서 PPT 파일 생성하기
### Assistants API
  - 작업을 대신 해주는 똑똑한 비서(Assistant)를 앱 안에 넣는 방법
  - 비서에게 모델, 규칙(설명서), 도구를 미리 달아주고, 대화(스레드)를 시작해서 일을 시킬 수 있음.
  - 대화 및 파일생성, 코드 명령 수행, 데이터 거맥 등 특정한 task에 특화되어 학습된 자연어 기반 생성형 AI API
- OpenAI의 일반적인 Chat AI처럼 대화형 서비스도 제공하지만, Assistants는 특정 작업에 특화하여 지원하거나 자동화에 중점을 둠으로써 챗봇, 가상비서, 자동응답시스템 개발 등에 필요한 작업들에 포커스를 맞춰서 활용 가능
- OpenAI platform의 API reference의 assistants 탭에서 더 많은 기능 확인 가능

In [3]:
from openai import OpenAI
from getpass import getpass
import time

In [5]:
MY_API_KEY = getpass.getpass("OpenAI API Key: ")

OpenAI API Key:  ········


In [43]:
client = OpenAI(api_key = MY_API_KEY)

## 1) assistant, thread, message, run 4가지 객체 생성
- assistant
  - 어떤 모델로, 어떤 규칙으로, 무슨 도구를 쓸지 정함.
  - client.beta.assistants.create(...)로 생성
- thread
  - Assistants API에서 하나의 대화 흐름을 관리하는 컨테이너 or 대화방(대화기록이 계속 쌓임.)
  - client.beta.threads.create()로 생성
- messsage
    - 사용자/비서가 남기는 실제 말
    - client.beta.threads.messages.create()로 생성
- run
  - 비서야, 이 대화(Thread)를 처리해! 하고 작동시키는 트리거. 비서는 필요하면 도구를 써서 결과를 만든다.
  - client.beta.threads.runs.create()로 생성
### 비서가 쓸 수 있는 도구❓❓
- Code Interpreter
  - 비서가 파이썬 코드를 직접 실행할 수 있음.
  - 데이터 분석, 차트 그리기, 그리고 파일 생성(CSV, 이미지, PDF 등) 도 가능
- File Search(지식 파일 검색)
  - 내가 올린 자료들(벡터 스토어)에 대해 찾아보고 근거를 뽑을 수 있음.
- Function Calling(함수 호출)
  - 비서가 네가 정의해둔 외부 함수(예: DB 조회, 사내 API 호출)를 필요할 때 골라 호출
### 비서를 왜 쓰는데❓❓
- 멀티스텝 자동화
  - 파일 받고 → 코드 돌리고 → 파일 만들어 돌려주기 같은 복잡한 과정을 비서가 스스로 처리
- 대화 상태 유지
  - Thread가 자동으로 히스토리를 관리하니, 프롬프트를 매번 길게 재전달할 필요가 줄어듦.
- 파일 입·출력 내장
  - 업로드한 파일을 읽고, 분석 결과를 파일로 만들어 돌려줄 수 있음.
### 진행 프로세스
  1. assistant 세부사항 정의
     - 어떤 모델을 쓸지
     - 프롬프트 작성
     - 필요한 도구 지정  
  2. assistant 객체 생성
  3. thread 및 message 객체 생성
  4. thread 실행(run)
     - 어떤 assistant를 쓸 건지 연결
     - run이 완료되면 메세지를 가져와서 비서의 응답 확인
     - 파일을 만들었다면 다운로드도 가능

### 1) assistant의 역할 및 전제 정의(role에서 system과 유사한 기능)

In [46]:
assistant_instruction = """
I need to make a PPT presentation file regarding Taylor Swift. You are an expert of the music industry and you exceed at creating PowerPoint files.
It should be written in precise fonts and in Korean. Each page needs a 40points big title, 20points big content. I need 3 slide in total. 
"""

# 원하는 요청에 대한 자연어 질문 작성
prompt_user = """
Make a presentation file about Taylor Swift. Explain concretely abount her career, music style, famous songs, and the reason why she's so famous.
"""

### .beta
- 베타(시험판) 영역이라는 뜻
- client.beta 아래에 있는 기능들은 새로운 API 버전(v2 등)이나 미리보기 기능을 쓰도록 붙여 둔 별도 통로
- 그래서 가끔 이름/동작이 바뀌거나 옮겨갈 수 있음.
-  Assistants(비서), Threads(대화방), Runs(실행), Code Interpreter 같은 에이전트형 기능이 여기에 묶여 있음!

### 2) assistant 객체 생성

In [48]:
assistant = client.beta.assistants.create(name = "My assistant", # 이름은 원하는대로 작성
                                          instructions = assistant_instruction, # 사전에 작성한 지침
                                          tools = [{"type":"code_interpreter"}], # Assistant API이 동작 방식 설정(code_interpreter, function, file_search 중에 하나)
                                          model = "gpt-4o")
assistant

Assistant(id='asst_gsYZVflbtURA5Etlr7uG1pSq', created_at=1754553431, description=None, instructions='\nI need to make a PPT presentation file regarding Taylor Swift. You are an expert of the music industry and you exceed at creating PowerPoint files.\nIt should be written in precise fonts and in Korean. Each page needs a 40points big title, 20points big content. I need 3 slide in total. \n', metadata={}, model='gpt-4o', name='My assistant', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0, reasoning_effort=None)

In [50]:
# 경고 미출력하는 코드
# 코드 실행에 꼭 필요한 건 아니지만 깔끔한 콘솔 유지를 위해 종종 넣음.
import warnings
warnings.filterwarnings(action = 'ignore')

### 3) thread 및 message 객체 생성
thread 객체 생성 = 대화방 생성

In [52]:
thread = client.beta.threads.create()

# 메세지 객체 생성
message = client.beta.threads.messages.create(thread_id = thread.id, # 이 메세지가 들어갈 대화방 지정
                                              role = "user", # 사람이 보낸 메세지라는 뜻
                                              content = prompt_user # 내가 쓴 Taylor Swift PPT 요청문
                                             )

### role: 누가 메세지를 보냈는지 표시
- user: 사람이 보냄.
- assistant: 비서(AI)가 보냄.
- system: 비서의 행동 규칙

### 4) run (=thread 실행)
- 이 대화(thread)를 내가 만든 assistant에게 실제로 처리하라고 지시하는 단계
- LLM으로 사용자의 요청이 넘어감.
- thread_id → 어떤 대화방을 처리할지 지정
- assistant_id → 어떤 비서를 쓸지 지정
- thread는 사용자가 요청을 입력했을 때 시작되고 모델의 응답이 끝나면 종료됨.
- 생성형 모델이 사용자의 요청을 인지하고 결과물을 생성하기 위해서는 어느정도의 시간이 필요함(복잡할수록 더욱 많은 시간이 필요함)

In [54]:
run = client.beta.threads.runs.create(thread_id = thread.id,
                                      assistant_id = assistant.id)

## assistant의 응답에 대한 진행상황 확인용 코드
- 이미 존재하는 객체를 다시 불러오는 역할
- 객체의 현재 상태나 정보를 다시 요청하는 메소드
- 매번 retrieve를 호출해서 run_retrieve.status 값을 확인
  - retrieve: 스레드의 실행 상태나 실행 후 결과를 검색하는데 사용되는 함수. client.beta.threads.runs.retrieve()로 실행. thread.id, run.id 필요
    - "in_progress" → 실행 중
    - "completed" → 완료됨
    - "failed" → 실패함
    - cancelled, expired 도 나올 수 있음.

In [58]:
while True:
    run_retrieve = client.beta.threads.runs.retrieve(thread_id = thread.id, # 스레드 객체의 id
                                                      run_id = run.id) # 스레드 실행 객체의 id
    # 실행완료인 경우 COMPLETE! 출력
    if run_retrieve.status == 'completed':
        print('COMPLETE!')
        break
    # 작업이 실행 중이거나 실패한 경우
    else:
        print(run_retrieve.status) # 해당 상태 출력
        time.sleep(3) # 3초 텀을 두고 상태를 계속해서 출력

in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
COMPLETE!


## thread 안에 있는 메세지(대화 내용)을 가져와서 원하는 부분만 뽑아서 살펴보기

In [60]:
# 특정 thread(대화방)에 쌓인 모든 메시지 목록을 가져옴
# message.list: 특정 스레드에서 오고간 메세지에 대한 정보를 출력하는 함수
messages = client.beta.threads.messages.list(thread_id = thread.id)
message

Message(id='msg_P4gHpO08OVE72D96y89xxrCa', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="\nMake a presentation file about Taylor Swift. Explain concretely abount her career, music style, famous songs, and the reason why she's so famous.\n"), type='text')], created_at=1754553435, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_wRPbsmPqP8088jxd88NTk7n9')

In [62]:
# 대화에서 가장 최근 메시지임. 보통 Assistant의 최신 응답
messages.data[-1]

Message(id='msg_P4gHpO08OVE72D96y89xxrCa', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="\nMake a presentation file about Taylor Swift. Explain concretely abount her career, music style, famous songs, and the reason why she's so famous.\n"), type='text')], created_at=1754553435, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_wRPbsmPqP8088jxd88NTk7n9')

In [64]:
# 첫 번째 메시지: 대화에서 가장 오래된 것, 보통 사용자가 처음 보낸 요청
# .text.value → 그 콘텐츠가 텍스트일 경우, 실제 문자열 내용을 꺼냄.
# 결과로 순수 텍스트 문자열이 나옴.
messages.data[0].content[0].text.value

"The third and final slide discussing the reasons for Taylor Swift's fame has been added to the presentation. You can download the complete PowerPoint presentation using the link below:\n\n[Taylor_Swift_Presentation.pptx](sandbox:/mnt/data/Taylor_Swift_Presentation.pptx)"

## annotations: 텍스트 안에서 특별한 태그·링크·참조 같은 부가정보 목록
- annotaions[0] → 그 중 첫 번째 annotation을 가져옴.
- annotations가 비어있으면 뭔가 잘못 된 거임.
- file_path=FilePath(....,file_id='file-6XShLZAFiwVXjJZeGsrUsV'): 파일의 OpenAI Files API 고유 ID. Assistant로 생성한 PPT 파일의 서버상의 식별자!!!✨

In [68]:
messages.data[0].content[0].text.annotations[0]

FilePathAnnotation(end_index=267, file_path=FilePath(file_id='file-6XShLZAFiwVXjJZeGsrUsV'), start_index=219, text='sandbox:/mnt/data/Taylor_Swift_Presentation.pptx', type='file_path')

## 5) 완료된 작업에 대한 정보 추출 및 실제 파일로 내보내기
- GPT가 만든 PPT 파일을 서버에서 꺼내서, 내 컴퓨터에 저장
- .file_path.file_id로 그 annotation이 가리키는 파일 ID 찾기
- client.files: 파일 관리
- .with_raw_response: 가공되지 않은 형태로 파일 받기
- .retrieve_content(file_id_path): 서버에서 그 파일의 실제 내용을 다운로드

In [72]:
file_id_path = messages.data[0].content[0].text.annotations[0].file_path.file_id
file_id_path
# assistant가 생성한 파일 id가 'file-6XShLZAFiwVXjJZeGsrUsV'임.

'file-6XShLZAFiwVXjJZeGsrUsV'

In [74]:
file_contents = client.files.with_raw_response.retrieve_content(file_id_path)
file_contents

<APIResponse [200 OK] type=<class 'str'>>

### <APIResponse [200 OK] type=<class 'str'>> 이게 뭔 뜻이냐
- 200 → 요청이 정상적으로 성공했다는 의미
- OK → 상태 코드에 붙는 설명(성공)
- type=<class 'str'>는 응답 본문이 문자열 형태로 저장돼 있다는 의미
- 근데 PPT, PDF 같은 바이너리 파일을 받을 때는 문자열이 아니라 **바이트(bytes)**로 다뤄야 안전
- 이 상태에서 바로 쓰기(f.write(...))를 하면, 바이너리 파일이 깨질 수 있음.

### .content를 사용하면, 바이트 데이터로 받을 수 있음!!!
👀 서버에서 받은 PPT 내용을 내 컴퓨터에 PPT_001.pptx라는 이름으로 저장하는 법:
- 내 컴퓨터에 PPT_001.pptx라는 파일을 **쓰기 모드(wb)**로 열기
- wb = write binary, PPT처럼 텍스트가 아닌 파일을 저장할 때 필요
- f.write(file_contents.content) → 다운로드해 온 파일 내용을 그대로 써서 저장

In [78]:
with open('PPT_001.pptx','wb') as f:
    f.write(file_contents.content)

## Recap
- 간단한 요청을 진행했기 때문에 결과물의 퀄리티는 높지 않지만 Assistant API를 활용하면 일반 대화형 API로 하기 힘든 파일 제작 등 특정 task에 특화된 응답과 결과를 얻을 수 있음.